## 10.1 构建一个模块的层级包

封装成包是很简单的。在文件系统上组织你的代码,并`确保每个目录都定义了一个
__init__.py 文件`。

定义模块的层次结构就像在文件系统上建立目录结构一样容易。文件 `__init__.py`
的目的是要`包含不同运行级别的包的可选的初始化代码`。举个例子,如果你执行了语句
import graphics,文件 graphics/\_\_init\_\_.py 将被导入, 建立 graphics 命名空间的内容。
像 import graphics.format.jpg 这样导入,文件 graphics/\_\_init\_\_.py 和文件 graphics/
formats/\_\_init\_\_.py 将在文件 graphics/formats/jpg.py 导入之前导入。

导入子模块时，父模块的`__init__.py`会在它的各个子模块的`__init__.py`之前执行

绝大部分时候让 `__init__.py` 空着就好。但是有些情况下可能包含代码。举个例
子,`__init__.py` 能够用来自动加载子模块:

`__init__.py`的其他常用用法包括将多个文件合并到一个逻辑命名空间

## 10.2 控制模块被全部导入的内容

### 目的：
当使用’from module import *’语句时,希望对从模块或包导出的符号进行精确控制。
在你的模块中定义一个变量 `__all__` 来明确地列出需要导出的内容。



In [2]:
# somemodule.py
def spam():
    pass
def grok():
    pass
blah = 42
# Only export 'spam' and 'grok'
__all__ = ['spam', 'grok']

## 10.3 使用相对路径名导入包中子模块

### 问题
将代码组织成包, 想用 import 语句从另一个包名没有硬编码过的包的中导入子模
块。

In [5]:
# mypackage/A/spam.py
from . import grok

In [ ]:
# mypackage/A/spam.py
from ..B import bar

两个 import 语句都没包含顶层包名,而是使用了 spam.py 的相对路径。

在包内,既可以使用相对路径也可以使用绝对路径来导入。

In [ ]:
# mypackage/A/spam.py
from mypackage.A import grok # OK
from . import grok # OK
import grok # Error (not found)

像 mypackage.A 这样使用绝对路径名的不利之处是这将顶层包名硬编码到你的源
码中。如果你想重新组织它,你的代码将更脆,很难工作。举个例子,如果你改变了包
名,你就必须检查所有文件来修正源码。同样,硬编码的名称会使移动代码变得困难。
举个例子,也许有人想安装两个不同版本的软件包,只通过名称区分它们。如果使用相
对导入,那一切都 ok,然而使用绝对路径名很可能会出问题。

## 10.4 将模块分割成多个文件

L**需求**： 你想将一个模块分割成多个文件。但是你不想将分离的文件统一成一个逻辑模块时使已有的代码遭到破坏。

**解决办法**: 将大文件拆开后，创建一个文件夹，将拆分成的小文件放置期内，在文件夹内建立一个`__init__.py`文件。这个`__init__.py`内容如下:

In [ ]:
# 文件夹内容
# mymodule/
#     __init__.py
#     a.py
#     b.py
# 
# __init__.py
from .a import A
from .b import B

## 10.5 利用命名空间导入目录分散的代码

在这里工作的机制被称为“包命名空间”的一个特征。从本质上讲,**包命名空间是
一种特殊的封装设计,为合并不同的目录的代码到一个共同的命名空间。**对于大的框
架,这可能是有用的,因为它允许一个框架的部分被单独地安装下载。它也使人们能够
轻松地为这样的框架编写第三方附加组件和其他扩展。

**包命名空间的关键是确保顶级目录中没有 `__init__.py` 文件来作为共同的命名空
间。**   
缺失 `__init__.py` 文件使得在导入包的时候会发生有趣的事情:  
这并没有产生错误,解释器创建了一个由所有包含匹配包名的目录组成的列表。特殊的包命名空间模块
被创建,只读的目录列表副本被存储在其 `__path__` 变量中。

一个包是否被作为一个包命名空间的主要方法是检查其 `__file__` 属性。如果没有,那包是个命名空间。

## 10.6 重新加载模块

源码进行了修改， 使用 `imp.reload()` 来重新加载先前加载的模块。

reload() 擦除了模块底层字典的内容,并通过重新执行模块的源代码来刷新它。

在生产环境中可能需要避免重新加载模块。

## 10.7 运行目录或压缩文件

如果你的应用程序已经有多个文件,你可以把你的应用程序放进它自己的目录并添加一个 `__main__.py` 文件。举

## 10.8 读取位于包中的数据文件

现在假设 spam.py 文件需要读取 somedata.dat 文件中的内容。你可以用以下代码
来完成:

In [ ]:
# spam.py
import pkgutil
data = pkgutil.get_data(__package__, 'somedata.dat')

要读取数据文件,你可能会倾向于编写使用内置的 I/ O 功能的代码,如 open()。**但是这种方法也有一些问题:**
- 首先,**一个包对解释器的当前工作目录几乎没有控制权.**因此,编程时任何 I/O 操作都必须使用绝对文件名。由于每个模块包含有完整路径的 __file__ 变量,这弄清楚它的路径不是不可能,但它很凌乱。
- 第二,**包通常安装作为.zip 或.egg 文件**,这些文件并不像在文件系统上的一个普通目录里那样被保存。因此,你试图用 open() 对一个包含数据文件的归档文件进行操作,它根本不会工作。

pkgutil.get_data() 函数是一个读取数据文件的高级工具,**不用管包是如何安装以及安装在哪**。它只是工作并将文件内容以字节字符串返回给你

get_data() 的第一个参数是包含包名的字符串。你可以直接使用包名,也可以使
用特殊的变量,比如 `__package__`。第二个参数是包内文件的相对名称。

## 10.9 将文件夹加入到 sys.path

### 第一种方法: 可以使用 `PYTHONPATH`环境变量来添加。

在自定义应用程序中,这样的环境变量可在程序启动时设置或通过 shell 脚本。

### 第二种方法是创建一个.pth 文件,将目录列举出来

这个.pth 文件需要放在某个 Python 的 site-packages 目录,通常位于/usr/local/
lib/python3.3/site-packages 或者 ~/.local/lib/python3.3/sitepackages。

当解释器启动时,.pth 文件里列举出来的存在于文件系统的目录将被添加到 sys.path。安装一个.pth 文件
可能需要管理员权限,如果它被添加到系统级的 Python 解释器。

*比起费力地找文件,你可能会倾向于写一个代码手动调节 sys.path 的值。*

In [ ]:
import sys
sys.path.insert(0, '/some/dir')
sys.path.insert(0, '/other/dir')

虽然这能“工作”,它是在`实践中极为脆弱,应尽量避免使用`。这种方法的问题是, **它将目录名硬编码到了你的源代码**。

`site-packages 目录是第三方包和模块安装的目录。`如果你手动安装你的代码,它将
被安装到 site-packages 目录。**虽然用于配置 path 的.pth 文件必须放置在 site-packages
里,但它配置的路径可以是系统上任何你希望的目录。因此,你可以把你的代码放在一
系列不同的目录,只要那些目录包含在.pth 文件里。**

## 10.10 通过字符串名导入模块

Q: 你想导入一个模块,但是模块的名字在字符串里。你想对字符串调用导入命令。

A: 使用 `importlib.import_module()` 函数来手动导入名字为字符串给出的一个模块或
者包的一部分。

In [ ]:
import importlib
math = importlib.import_module('math')
>>> math.sin(2)
0.9092974268256817
>>> mod = importlib.import_module('urllib.request')
>>> u = mod.urlopen('http://www.python.org')
>>>

如果你正在使用的包,import_module()个额外的参数。例如:也可用于相对导入。需要给它传递一个额外的参数。

In [ ]:
import importlib
# Same as 'from . import b'
b = importlib.import_module('.b', __package__)